In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time

In [2]:
def chi2(a,b,c,d):
    if a+b == 0:
        return (c-d)**2/(c+d)
    elif c+d == 0:
        return (a-b)**2/(a+b)
    else:
        return ((a-b)**2)/(a+b) + ((c-d)**2)/(c+d)

In [3]:
def LS(a,b,c,d): #local sensitivity
    stat = chi2(a,b,c,d); t = np.zeros(0)
    M = chi2(a,b,c,d); m = chi2(a,b,c,d)
    if a >= 2:
        t = [chi2(a-2,b+2,c,d), chi2(a-2,b+1,c+1,d), chi2(a-2,b+1,c,d+1), chi2(a-2,b,c+2,d), chi2(a-2,b,c+1,d+1),
             chi2(a-2,b,c,d+2)]
        M = max([max(t),M])
        m = min([min(t),m])
    if b >= 2:
        t = [chi2(a+2,b-2,c,d), chi2(a+1,b-2,c+1,d), chi2(a+1,b-2,c,d+1), chi2(a,b-2,c+2,d), chi2(a,b-2,c+1,d+1),
             chi2(a,b-2,c,d+2)]
        M = max([max(t),M])
        m = min([min(t),m])
    if c >= 2:
        t = [chi2(a+2,b,c-2,d), chi2(a+1,b+1,c-2,d), chi2(a+1,b,c-2,d+1), chi2(a,b+2,c-2,d), chi2(a,b+1,c-2,d+1),
             chi2(a,b,c-2,d+2)]
        M = max([max(t),M])
        m = min([min(t),m])
    if d >= 2:
        t = [chi2(a+2,b,c,d-2), chi2(a+1,b+1,c,d-2), chi2(a+1,b,c+1,d-2), chi2(a,b+2,c,d-2), chi2(a,b+1,c+1,d-2),
             chi2(a,b,c+2,d-2)]
        M = max([max(t),M])
        m = min([min(t),m])
    if a >= 1 and b >= 1:
        t = [chi2(a-1,b-1,c+2,d), chi2(a-1,b-1,c+1,d+1), chi2(a-1,b-1,c,d+2)]
        M = max([max(t),M])
        m = min([min(t),m])
    if a >= 1 and c >= 1:
        t = [chi2(a-1,b+2,c-1,d), chi2(a-1,b+1,c-1,d+1), chi2(a-1,b,c-1,d+2)]
        M = max([max(t),M])
        m = min([min(t),m])
    if a >= 1 and d >= 1:
        t = [chi2(a-1,b+2,c,d-1), chi2(a-1,b+1,c+1,d-1), chi2(a-1,b,c+2,d-1)]
        M = max([max(t),M])
        m = min([min(t),m])
    if b >= 1 and c >= 1:
        t = [chi2(a+2,b-1,c-1,d), chi2(a+1,b-1,c-1,d+1), chi2(a,b-1,c-1,d+2)]
        M = max([max(t),M])
        m = min([min(t),m])
    if b >= 1 and d >= 1:
        t = [chi2(a+2,b-1,c,d-1), chi2(a+1,b-1,c+1,d-1), chi2(a,b-1,c+2,d-1)]
        M = max([max(t),M])
        m = min([min(t),m])
    if c >= 1 and d >= 1:
        t = [chi2(a+2,b,c-1,d-1), chi2(a+1,b+1,c-1,d-1), chi2(a,b+2,c-1,d-1)]
        M = max([max(t),M])
        m = min([min(t),m])
    if a >= 1:
        t = [chi2(a-1,b+1,c,d), chi2(a-1,b,c+1,d), chi2(a-1,b,c,d+1)]
        M = max([max(t),M])
        m = min([min(t),m])
    if b >= 1:
        t = [chi2(a+1,b-1,c,d), chi2(a,b-1,c+1,d), chi2(a,b-1,c,d+1)]
        M = max([max(t),M])
        m = min([min(t),m])
    if c >= 1:
        t = [chi2(a+1,b,c-1,d), chi2(a,b+1,c-1,d), chi2(a,b,c-1,d+1)]
        M = max([max(t),M])
        m = min([min(t),m])
    if d >= 1:
        t = [chi2(a+1,b,c,d-1), chi2(a,b+1,c,d-1), chi2(a,b,c+1,d-1)]
        M = max([max(t),M])
        m = min([min(t),m])
    
    return max([M-stat, stat-m])

In [4]:
def SS(table,beta): #smooth sensitivity
    a = table[0]; b = table[1]; c = table[2]; d = table[3]
    GS = 8; ls = LS(a,b,c,d)
    if beta >= math.log(3):
        return ls
    else:
        dist = math.ceil(math.log(GS/ls))/beta
        t = int(2*dist); ss = 0
        for ya in range(-t,t+1):
            for yb in range(-t,t+1):
                for yc in range(-t,t+1):
                    yd = -ya-yb-yc
                    if a+ya >= 0 and b+yb>=0 and c+yc>=0 and d+yd>=0:
                        dxy = math.ceil((math.fabs(ya) + math.fabs(yb) + math.fabs(yc) + math.fabs(yd))/4)
                        if dxy <= dist:
                            ss = max([LS(a+ya,b+yb,c+yc,d+yd)*math.exp(-(beta*dxy)), ss])
        return ss

In [5]:
def generateData(n):
    t = int(2*n); table = np.zeros(4)
    table[0] = np.random.binomial(t,1/4)
    table[1] = np.random.binomial(t-table[0],1/3)
    table[2] = np.random.binomial(t-table[0]-table[1],1/2)
    table[3] = t-table[0]-table[1]-table[2]
    return table

In [6]:
def RunTime(N,beta):
    t = 0
    for i in range(100):
        table = generateData(N)
        s = time.time()
        SS(table,beta)
        e = time.time()
        t += (e-s)
    return t/100

In [7]:
N = 1000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

1.0020314455032349
0.14325281381607055
6.667375564575195e-05
6.648540496826172e-05


In [8]:
N = 2000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

1.3569517183303832
0.17910594940185548
6.708145141601563e-05
6.65426254272461e-05


In [9]:
N = 5000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

1.8278893518447876
0.23533127784729005
6.86788558959961e-05
6.63614273071289e-05
